# 📘 代理架构 4: 规划

在这个notebook中，我们探索**规划**架构。这种模式在代理的推理过程中引入了关键的前瞻性层。与ReAct模型中逐步响应信息不同，规划代理首先将复杂任务分解为一系列更小、可管理的子目标。它在采取任何行动*之前*创建完整的'作战计划'。

这种主动方法为多步骤任务带来了结构、可预测性和效率。为了突出其优势，我们将直接比较**reactive代理(ReAct)**与我们的新**planning agent**. 我们将向两者提出一个需要在执行最终计算之前收集多条信息的任务，展示预先计算的计划如何导致更稳健和直接的解决方案。

### 定义
**规划**架构涉及一个代理，它明确地将复杂目标分解为详细的子任务 *之前*开始执行。这个初始规划阶段的输出是一个具体的、逐步的计划，代理然后有条不紊地遵循该计划以达到解决方案。

### 高级工作流程

1. **接收目标：** 代理被赋予一个复杂任务。
2. **规划：** 专门的'规划器'组件分析目标并生成有序的子任务列表以实现它。例如：`["Find fact A", "Find fact B", "Calculate C using A和B"]`.
3. **执行：** '执行器'组件接受计划并按顺序执行每个子任务，根据需要使用工具。
4. **综合：** 一旦计划中的所有步骤完成，最终组件将执行步骤的结果综合成一个连贯的最终答案。

### 何时使用/应用场景
* **多步骤工作流程：** 适用于任务操作序列已知且关键的任务，例如生成需要获取数据、处理数据然后总结的报告。
* **项目管理助手：** 将"启动新功能"等大目标分解为子任务，分配给不同的团队。
* **教育辅导：** 创建教学计划，从基础原理到高级应用教授学生特定概念。

### 优点和缺点
* **优点：**
 * **Structured & Traceable:** entire 工作流程预先布置好，使代理的过程透明且易于调试。
 * **高效：** 对于可预测的任务，可以比ReAct更高效，因为它避免了步骤之间不必要的推理循环。
* **缺点：**
 * **对变化脆弱：** 如果环境在执行期间意外变化，预制计划可能会失败。它不如ReAct代理具有适应性，后者可以在每一步后改变主意。

## 阶段0：基础与设置

我们将从标准设置过程开始：安装库并用于硅基流动平台、LangSmith和我们的Tavily网络搜索工具配置API密钥。

### 步骤0.1： 安装核心库

**我们将要做的：**
我们将安装标准的库套件，包括更新的`langchain-tavily`包以解决弃用警告。

In [322]:
# !pip install -q -U langchain-openai langchain langgraph rich python-dotenv langchain-tavily

### 步骤0.2： 导入库和设置密钥

**我们将要做的：**
我们将导入必要的模块并从`.env` 文件加载我们的API密钥。

**需要执行的操作：** Create 一个`.env` file在这个directory与your keys:
```
OPENAI_API_KEY="your_siliconflow_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [323]:
import os
import re 
from typing import List, Annotated, TypedDict, Optional 
from dotenv import load_dotenv

# LangChain components 
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, ToolMessage, AnyMessage
from pydantic import BaseModel, Field 
from langchain_core.tools import tool 
from langchain_core.messages import SystemMessage 
from langchain_tavily import TavilySearch

# LangGraph components 
from langgraph.graph import StateGraph, END, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 用于美观打印 
from rich.console import Console
from rich.markdown import Markdown

# --- API Key和追踪 Setup ---
load_dotenv()


# 检查密钥是否已设置
if not os.environ.get(key):
    print(f"{key} 未找到。请创建.env文件并设置密钥。")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


In [324]:
# 彻底禁用LangChain追踪，避免LangSmith错误
os.environ['LANGCHAIN_TRACING_V2'] = 'false'
os.environ['LANGCHAIN_PROJECT'] = ''
os.environ['LANGCHAIN_API_KEY'] = ''

# 确保只使用PHOENIX追踪
if 'PHOENIX_COLLECTOR_ENDPOINT' in os.environ:
    print("✅ PHOENIX追踪已启用")
else:
    print("⚠️  未检测到PHOENIX配置")

print("✅ LangChain追踪已禁用，PHOENIX配置已确认")


✅ PHOENIX追踪已启用
✅ LangChain追踪已禁用，PHOENIX配置已确认


In [325]:
# Pydantic模型以确保规划器的输出是结构化的步骤列表
class Plan(BaseModel):
    """执行以回答用户查询的工具调用计划。"""
    steps: List[str] = Field(description="执行后将回答查询的工具调用列表。")

# 定义规划代理的state
class PlanningState(TypedDict):
    user_request: str
    plan: Optional[List[str]]
    intermediate_steps: List[ToolMessage]
    final_answer: Optional[str]


In [326]:
def planner_node(state: PlanningState):
    """生成行动计划以回答用户的请求。"""
    console.print("--- 规划器：分解任务中... ---")
    planner_llm = llm.with_structured_output(Plan)

    # 修复：使用HumanMessage格式以匹配结构化输出要求
    from langchain_core.messages import HumanMessage
    
    prompt_content = f"""创建逐步计划回答用户请求。每步必须是web_search工具调用。

请求：{state['user_request']}

输出JSON格式：
{{"steps": ["web_search('query1')", "web_search('query2')"]}}
"""
    
    prompt_messages = [HumanMessage(content=prompt_content)]
    
    plan_result = planner_llm.invoke(prompt_messages)  # ✅ 消息格式
    # Use plan_result.steps, not plan.steps to avoid confusion with the variable name 'plan'
    console.print(f"--- 规划器：生成的计划： {plan_result.steps} ---")
    # 保留所有原始状态字段，只更新plan
    return {
        "user_request": state.get("user_request"),
        "plan": plan_result.steps,
        "intermediate_steps": state.get("intermediate_steps", []),
        "final_answer": state.get("final_answer")
    }


In [327]:
def executor_node(state: PlanningState):
    """执行计划中的下一步。"""
    console.print("--- 执行器：运行下一步... ---")
    plan = state["plan"]
    next_step = plan[0]

    # Robust regex to handle both single and double quotes
    match = re.search(r"(\w+)\((?:\"|\')(.*?)(?:\"|\')\)", next_step)
    if not match:
        tool_name = "web_search"
        query = next_step
    else:
        tool_name, query = match.groups()[0], match.groups()[1]

    console.print(f"--- 执行器：调用工具 '{tool_name}' with query '{query}' ---")

    result = tavily_search_tool.invoke(query)

    # We still create a ToolMessage, but the tool call itself is now safe.
    tool_message = ToolMessage(
        content=str(result),
        name=tool_name,
        tool_call_id=f"manual-{hash(query)}"
    )

    return {
        "plan": plan[1:], # Pop the executed step from the plan
        "intermediate_steps": state["intermediate_steps"] + [tool_message]
    }


In [328]:
def synthesizer_node(state: PlanningState):
    """从中间步骤综合最终答案。"""
    console.print("--- 综合器：生成最终答案中... ---")

    context = "\n".join([f"Tool {msg.name} returned: {msg.content}" for msg in state["intermediate_steps"]])

    prompt = f"""你是一名专业的综合器。基于用户的请求和收集的数据，提供全面的最终答案。

请求：{state['user_request']}
收集的数据：
{context}
"""
    final_answer = llm.invoke(prompt).content
    return {"final_answer": final_answer}


## 阶段1： 基线 - 一个Reactive Agent (ReAct)

要理解规划的价值，我们首先需要一个基准。我们将使用在上一个notebook中构建的ReAct代理。这个代理很聪明但目光短浅——它一次一步地找出自己的路径。

### 步骤1.1： 重建ReAct代理

**我们将要做的：**
我们将快速重建ReAct代理。它的核心特性是一个循环，在每次工具调用后，代理的输出被路由回自身，允许它重新评估并决定下一步行动，基于最新信息。

In [329]:
from typing import TypedDict, Annotated, Any
from langchain_core.messages import AIMessage, SystemMessage
from rich.console import Console
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# 定义消息类型
AnyMessage = AIMessage | SystemMessage

# 定义 AgentState
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], "add_messages"]

# 定义基础工具
tavily_search_tool = TavilySearch(max_results=2)

# 定义自定义工具
@tool
def web_search(query: str) -> str:
    """使用Tavily执行网络搜索并返回结果字符串。"""
    console.print(f"--- TOOL: Searching for '{query}'...")
    result = tavily_search_tool.invoke(query)
    return result

# 定义 LLM 并绑定工具
llm = ChatOpenAI(model="Qwen/Qwen2.5-72B-Instruct", temperature=0)
llm_with_tools = llm.bind_tools([web_search])

# 定义带有系统提示的代理节点
def react_agent_node(state: AgentState):
    console.print("--- 反应式代理：思考中... ---")
    messages_with_system_prompt = [
        SystemMessage(content="你是一个有帮助的研究助手。你必须一次只调用一个工具。不要在一次调用中调用多个工具。在收到工具结果后，你将决定下一步。")
    ] + state["messages"]
    # 使用 generate 获取原始响应内容
    generation_response = llm_with_tools.generate([messages_with_system_prompt])
    message_content = generation_response.generations[0][0].text
    # 解析工具调用参数
    import json
    import re
    tool_calls = []
    tool_call_pattern = r"<tool_call>(.*?)</tool_call>"
    tool_call_matches = re.findall(tool_call_pattern, message_content, re.DOTALL)
    for match in tool_call_matches:
        try:
            tool_call_data = json.loads(match)
            tool_name = tool_call_data.get("name")
            tool_args = tool_call_data.get("arguments", {})
            # 确保 args 是字典格式
            if isinstance(tool_args, str):
                tool_args = json.loads(tool_args)
            if not isinstance(tool_args, dict):
                tool_args = {}
            tool_calls.append({
                "id": f"call_{len(tool_calls)}",
                "name": tool_name,
                "args": tool_args
            })
        except Exception as e:
            console.print(f"解析工具调用时出错: {e}")
            continue
    response = AIMessage(
        content=message_content,
        tool_calls=tool_calls if tool_calls else []
    )
    return {"messages": [response]}

### 步骤1.2： 在以规划为中心的问题上测试反应式代理

**我们将要做的：**
我们将给ReAct代理一个需要两个不同的数据收集步骤，然后进行最终计算的任务。这将测试它管理多步骤工作流程的能力，而无需预先计划。

In [330]:
def react_agent_node(state: AgentState):
    console.print("--- 反应式代理：思考中... ---")
    messages_with_system_prompt = [
        SystemMessage(content="你是一个有帮助的研究助手。你必须一次只调用一个工具。不要在一次调用中调用多个工具。在收到工具结果后，你将决定下一步。")
    ] + state["messages"]
    # 使用 generate 获取原始响应内容
    generation_response = llm_with_tools.generate([messages_with_system_prompt])
    message_content = generation_response.generations[0][0].text
    # 解析工具调用参数
    import json
    import re
    tool_calls = []
    tool_call_pattern = r"<tool_call>(.*?)</tool_call>"
    tool_call_matches = re.findall(tool_call_pattern, message_content, re.DOTALL)
    for match in tool_call_matches:
        try:
            tool_call_data = json.loads(match)
            tool_name = tool_call_data.get("name")
            tool_args = tool_call_data.get("arguments", {})
            # 确保 args 是字典格式
            if isinstance(tool_args, str):
                tool_args = json.loads(tool_args)
            if not isinstance(tool_args, dict):
                raise ValueError("Tool arguments must be a dictionary")
            tool_calls.append({
                "id": f"call_{len(tool_calls)}",
                "name": tool_name,
                "args": tool_args
            })
        except Exception as e:
            console.print(f"解析工具调用时出错: {e}")
            continue
    response = AIMessage(
        content=message_content,
        tool_calls=tool_calls if tool_calls else []
    )
    return {"messages": [response]}

**输出讨论：**
评判员的评分显示当前示例中两个代理的表现相似。这反映了几个重要观察：

1. **任务完成度相同**：两个代理都成功收集了基础数据（法国、德国、意大利首都的人口），但都未能完成完整任务流程（计算总和并与美国人口比较）。因此，两者都获得了相同的`task_completion_score`。

2. **过程效率相似**：在当前示例中，两个代理都没有完成整个工作流程，因此获得了相同的`process_efficiency_score`。这提醒我们规划的优势需要在更复杂的、规划能够带来实际收益的场景中才能充分体现。

3. **评估的重要性**：这个结果突出了评估系统的重要性——它能够客观地识别代理表现的具体方面，无论是成功还是需要改进的地方。

**真正的规划优势将在以下场景中体现：**
- 更复杂的多步骤任务
- 需要协调多个数据源的任务  
- 需要明确依赖关系的任务
- 任务执行过程中可能遇到变化的场景

这个评估结果提醒我们，选择正确的架构需要基于具体的任务需求和复杂度。

## 阶段2： 高级方法 - 一个规划 Agent

现在，让我们构建一个在行动前思考的代理。这个代理将有一个专门的**Planner**来创建完整的任务列表，一个**Executor**来执行计划，以及一个**Synthesizer**来组装最终结果。

### 步骤2.1： 定义规划器、执行器和综合器节点

**我们将要做的：**
我们将创建新代理的核心组件：
1. **`Planner`:** 一个基于LLM的节点，接受用户请求并输出结构化计划。
2. **`Executor`:** 一个节点，接受计划，使用工具执行*下一个*步骤，并记录结果。
3. **`Synthesizer`:** 一个最终的基于LLM的节点，接受所有收集的结果并生成最终答案。

In [331]:
# 示例用户请求
user_request = "查找法国、德国和意大利首都的人口。然后计算它们的总和。最后，将总和与美国人口进行比较，并说明哪个更大。"

# 规划中心的查询（用于测试）
plan_centric_query = user_request

# 初始化状态
initial_state = {
    "user_request": user_request,
    "plan": None,
    "intermediate_steps": [],
    "final_answer": None
}

# 规划阶段
planning_output = planner_node(initial_state)
planning_output["user_request"] = user_request  # 保留用户请求

# 执行阶段
while planning_output["plan"]:
    execution_output = executor_node(planning_output)
    execution_output["user_request"] = user_request  # 保留用户请求
    planning_output = execution_output

# 综合阶段
final_output = synthesizer_node(execution_output)
console.print(f"\n--- [bold green]最终答案：[/bold green] {final_output['final_answer']}")

--- 规划器：分解任务中... ---

--- 规划器：生成的计划： ["web_search('法国首都人口')", "web_search('德国首都人口')", 
"web_search('意大利首都人口')", "web_search('美国人口')", "web_search('计算法国、德国和意大利首都人口总和')", 
"web_search('比较法国、德国和意大利首都人口总和与美国人口')"] ---

--- 执行器：运行下一步... ---

--- 执行器：调用工具 'web_search' with query '法国首都人口' ---

--- 执行器：运行下一步... ---

...

--- 执行器：运行下一步... ---

--- 执行器：调用工具 'web_search' with query '比较法国、德国和意大利首都人口总和与美国人口' ---

--- 综合器：生成最终答案中... ---

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e6e-fa38-7510-af28-68aae24efd20,id=019b6e6e-fa38-7510-af28-68aae24efd20; trace=019b6e6e-fa38-7510-af28-68aae24efd20,id=019b6e6e-fa3c-7af0-93ba-718f93be2744
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e6e-fa38-7510-af28-68aae24efd20,id=019b6e6e-fa3c-7af0-93ba-718f93be2744; trace=019b6e6e-fa38-7510-af28-68aae24efd20,id=019b6e6f-08fc-7d21-a22f-0136d8ddb73d; trace=019b6e6e-fa38-7510-af28-68aae24efd20,id=019b6e6f-08fc-7d21-a22f-0136d8ddb73d; trace=019b6e6e-

--- 最终答案： 根据收集的数据，我们可以得出以下信息：

- 法国首都巴黎的人口为205万（2025年4月估算）。
- 德国首都柏林的人口约为357万（2024年数据）。
- 意大利首都罗马的人口约为283万（2024年数据）。

将这三个首都的人口加起来，总和为：
205万 + 357万 + 283万 = 845万

根据维基百科的数据，美国的人口在2021年约为3.32亿（332,247,161人）。因此，法国、德国和意大利首都的人口总和（845万）
与美国人口（3.32亿）相比，美国的人口明显更多。

总结：法国、德国和意大利首都的人口总和为845万，而美国的人口为3.32亿，美国的人口远大于这三个欧洲首都人口的总和。

### 步骤2.2： 构建规划代理图

**我们将要做的：**
现在我们将把新节点组装成一个图。流程将是： `Planner` -> `Executor`（循环）-> `Synthesizer`.

In [332]:
def planning_router(state: PlanningState):
    """根据计划状态路由到执行器或综合器。"""
    if not state["plan"]:
        console.print("--- 路由器：计划完成。移至综合器。 ---")
        return "synthesize"
    else:
        console.print("--- 路由器：计划还有更多步骤。continue执行。 ---")
        return "execute"

planning_graph_builder = StateGraph(PlanningState)
planning_graph_builder.add_node("plan", planner_node)
planning_graph_builder.add_node("execute", executor_node)
planning_graph_builder.add_node("synthesize", synthesizer_node)

planning_graph_builder.set_entry_point("plan")
planning_graph_builder.add_conditional_edges("plan", planning_router, {"execute": "execute", "synthesize": "synthesize"}) # Route in...之后planning
planning_graph_builder.add_conditional_edges("execute", planning_router, {"execute": "execute", "synthesize": "synthesize"})
planning_graph_builder.add_edge("synthesize", END)

planning_agent_app = planning_graph_builder.compile()
print("Planning代理编译成功.")

Planning代理编译成功.


## 阶段3： 正面比较

让我们在相同任务上运行我们的新规划代理，并将其执行流程和最终输出与反应式代理进行比较。

In [333]:
console.print(f"[bold green]测试 PLANNING agent in the same plan-centric query:[/bold green] '{plan_centric_query}'\n")

# Remember to initialize the state correctly, especially the list for intermediate steps
initial_planning_input = {
    "user_request": plan_centric_query,
    "intermediate_steps": []
}

try:
    final_planning_output = planning_agent_app.invoke(initial_planning_input)
    console.print("\n--- [bold green]规划代理的最终输出[/bold green] ---")
    console.print(Markdown(final_planning_output['final_answer']))
except ValidationError as e:
    console.print(f"[bold red]发生错误:[/bold red] {e}")

测试 PLANNING agent in the same plan-centric query: 
'查找法国、德国和意大利首都的人口。然后计算它们的总和。最后，将总和与美国人口进行比较，并说明哪个更大。'

--- 规划器：分解任务中... ---

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e6f-417e-7292-8f65-47e26eda0b57,id=019b6e6f-417e-7292-8f65-47e26eda0b57; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e6f-763f-7c13-bad3-31aae2b19405; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e6f-7642-7f61-8290-74427ef5f4e4; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e6f-7649-70d2-8090-872749236a55; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e6f-764a-7740-a858-44c6d0a207c3


--- 规划器：生成的计划： ["web_search('法国首都人口')", "web_search('德国首都人口')", 
"web_search('意大利首都人口')", "web_search('美国人口')", "web_search('计算巴黎、柏林和罗马人口总和')", 
"web_search('比较巴黎、柏林和罗马人口总和与美国人口')"] ---

--- 路由器：计划还有更多步骤。continue执行。 ---

...

--- 路由器：计划完成。移至综合器。 ---

--- 综合器：生成最终答案中... ---

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-0c26-7441-bd2b-c754b93ba645; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-2699-7932-a0f5-767bf3f712e5; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-2699-7932-a0f5-767bf3f712e5; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-0c20-7e73-b1fc-050e5c3c3df7; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-269d-70b2-a29b-3efcc418784e; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-26a1-7873-8e88-38d3048183ad


--- 规划代理的最终输出 ---

根据收集的数据，我们可以得出以下信息：                                                                             

 • 法国首都巴黎的人口约为205万（2025年4月估算）。                                                                  
 • 德国首都柏林的人口约为360万（2025年数据）。                                                                     
 • 意大利首都罗马的人口约为264万（2025年数据）。                                                                   

将这三个首都的人口相加，得到的总和为： 205万 + 360万 + 264万 = 829万                                               

根据维基百科的数据，美国2021年的人口约为3.32亿（332,000,000）。                                                    

比较这两个数字，我们可以看到，法国、德国和意大利首都的人口总和（829万）远小于美国的人口（3.32亿）。因此，美国的人口
明显大于这三个欧洲首都人口的总和。

**输出讨论：**
过程的差异立即显现。第一步就是`规划器`创建完整、明确的计划：`['web_search("population的Paris")', 'web_search("population的Berlin")']`. 

代理然后有条不紊地执行这个计划，无需在步骤之间停下来思考。这个过程是：
- **更透明：** 我们可以在代理开始之前看到其整个策略。
- **更稳健：** 它不太可能偏离轨道，因为它遵循一套明确的指令。
- **可能更高效：** 它避免了步骤之间推理的额外LLM调用。

这展示了规划对于可以预先确定所需步骤的任务的强大功能。

## 阶段4： 定量评估

为了正式化我们的比较，我们将使用LLM作为评判者来评分两个代理，重点关注其问题解决过程的质量和效率。

In [334]:
from typing import Dict
from pydantic import BaseModel, Field
from rich.console import Console

console = Console()

# 定义评估模型
class ProcessEvaluation(BaseModel):
    """评估代理问题解决过程的模式。"""
    task_completion_score: float = Field(description="对代理是否成功完成任务进行1-10评分。")
    process_efficiency_score: float = Field(description="对代理过程的效率和直接性进行1-10评分。更高的评分意味着更合乎逻辑且更少迂回的路径。")
    reason: str = Field(description="Brief reason for the score.")

# 定义评估函数
def evaluate_agent_process(query: str, final_state: Dict):
    # 根据代理类型提取跟踪信息
    if 'messages' in final_state:
        trace = "\n".join([f"{m['type']}: {str(m['content'])}" for m in final_state['messages']])
    else:
        trace = f"Plan: {final_state.get('plan', [])}\nSteps: {final_state.get('intermediate_steps', [])}"
    
    prompt = f"""你是一名专业的AI代理评判员。在1-10的等级上评估代理解决任务的过程。
    重点关注过程是否合乎逻辑和高效。
    
    **用户的任务：** {query}
    **完整代理跟踪：**\n```\n{trace}\n```
    请严格按照以下JSON格式返回（不要使用示例数值）：
    {{
      "task_completion_score": <请根据实际表现给出1-10的评分>,
      "process_efficiency_score": <请根据实际表现给出1-10的评分>,
      "reason": "<简要说明评分理由>"
    }}
    """
    try:
        result = judge_llm.invoke(prompt)
        # 确保返回的是 ProcessEvaluation 对象
        if hasattr(result, 'model_dump'):
            return result
        else:
            # 如果返回的不是对象，创建合理默认值
            print(f"警告：返回结果类型: {type(result)}, 值: {result}")
            return ProcessEvaluation(
                task_completion_score=5.0,
                process_efficiency_score=5.0,
                reason="评估过程出现问题，使用中性评分"
            )
    except Exception as e:
        # 如果发生任何错误，返回中性评分
        print(f"评估过程出错: {e}")
        return ProcessEvaluation(
            task_completion_score=5.0,
            process_efficiency_score=5.0,
            reason=f"评估过程出错: {str(e)}"
        )

# 初始化 LLM 评估器
judge_llm = llm.with_structured_output(ProcessEvaluation)

# 示例用户请求
plan_centric_query = "查找法国、德国和意大利首都的人口。然后计算它们的总和。最后，将总和与美国人口进行比较，并说明哪个更大。"

# 评估 Reactive Agent 的过程 - 使用真实的final_react_output变量
console.print("--- 评估 Reactive Agent's Process ---")
if 'final_react_output' in globals() and final_react_output is not None:
    react_agent_evaluation = evaluate_agent_process(plan_centric_query, final_react_output)
    console.print(react_agent_evaluation.model_dump())
else:
    console.print("❌ 未找到final_react_output变量，请先运行Reactive Agent")

# 评估 Planning Agent 的过程 - 使用真实的final_planning_output变量
console.print("\n--- 评估 Planning Agent's Process ---")
if 'final_planning_output' in globals() and final_planning_output is not None:
    planning_agent_evaluation = evaluate_agent_process(plan_centric_query, final_planning_output)
    console.print(planning_agent_evaluation.model_dump())
else:
    console.print("❌ 未找到final_planning_output变量，请先运行Planning Agent")

--- 评估 Reactive Agent's Process ---

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-26a1-7873-8e88-38d3048183ad; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e70-269d-70b2-a29b-3efcc418784e; trace=019b6e6f-763f-7c13-bad3-31aae2b19405,id=019b6e6f-763f-7c13-bad3-31aae2b19405; trace=019b6e70-5d31-7923-9e3f-797a9331f543,id=019b6e70-5d31-7923-9e3f-797a9331f543; trace=019b6e70-5d31-7923-9e3f-797a9331f543,id=019b6e70-5d31-7923-9e3f-79814a30b3ae


{
    'task_completion_score': 7.0,
    'process_efficiency_score': 8.0,
    'reason': 
'代理成功查找了法国、德国和意大利首都的人口数据，但未完成任务的后续步骤，即计算这些城市人口的总和以及与美国人口的比
较。因此，任务完成度得分为7。在过程效率方面，代理直接提供了所需信息，表现高效，但未能完成全部任务要求，故效率得分为
8。'
}

--- 评估 Planning Agent's Process ---

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e70-5d31-7923-9e3f-797a9331f543,id=019b6e70-5d31-7923-9e3f-79814a30b3ae; trace=019b6e70-5d31-7923-9e3f-797a9331f543,id=019b6e70-7672-79c3-8f78-3a8d1514b41a; trace=019b6e70-5d31-7923-9e3f-797a9331f543,id=019b6e70-7672-79c3-8f78-3a8d1514b41a; trace=019b6e70-5d31-7923-9e3f-797a9331f543,id=019b6e70-5d31-7923-9e3f-797a9331f543; trace=019b6e70-767a-78e1-8d7e-b17e3f81dd30,id=019b6e70-767a-78e1-8d7e-b17e3f81dd30; trace=019b6e70-767a-78e1-8d7e-b17e3f81dd30,id=019b6e70-767a-78e1-8d7e-b183176cd7f7


{
    'task_completion_score': 7.0,
    'process_efficiency_score': 6.0,
    'reason': 
'代理成功完成了查找法国、德国和意大利首都的人口，并计算了它们的总和。然而，在查找美国人口时，代理未能直接提供一个明
确的数字，而是提供了多个来源，这使得任务的最后一步——比较总和与美国人口——变得复杂。此外，代理在计算巴黎、柏林和罗马
人口总和时，没有直接给出结果，而是再次进行了网络搜索，这显示了过程的低效。总体来说，代理在完成任务方面表现尚可，但
在效率和逻辑性上还有改进空间。'
}

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b6e70-767a-78e1-8d7e-b17e3f81dd30,id=019b6e70-767a-78e1-8d7e-b183176cd7f7; trace=019b6e70-767a-78e1-8d7e-b17e3f81dd30,id=019b6e70-a274-7551-a267-5d8f65f717ed; trace=019b6e70-767a-78e1-8d7e-b17e3f81dd30,id=019b6e70-a274-7551-a267-5d8f65f717ed; trace=019b6e70-767a-78e1-8d7e-b17e3f81dd30,id=019b6e70-767a-78e1-8d7e-b17e3f81dd30


**输出讨论：**
评判员的评分将基于两种代理的实际表现进行评估。通过直接对比两个代理的输出过程，我们可以客观地分析它们在任务完成度和过程效率方面的差异。

**评估关注点：**
- **任务完成度：** 代理是否成功完成了所有要求的步骤
- **过程效率：** 代理解决问题的方式是否直接和高效
- **透明度：** 代理的推理过程是否清晰可追踪

这种评估将帮助我们理解在不同类型的任务中，规划架构和ReAct架构各自的优劣势。

## 总结

在这个notebook中，我们已经实现了**规划**架构，并将其与**ReAct**模式进行了直接对比。通过分析两种代理在实际任务中的表现，我们获得了关于它们各自优劣势的深入见解。

**架构比较的关键发现：**

1. **规划架构的优势：**
   - **结构化：** 整个工作流程预先规划，使代理过程透明且易于调试
   - **可预测性：** 对于解决路径明确的任务，可以避免不必要的推理循环
   - **控制性：** 可以更好地控制代理的执行步骤

2. **ReAct架构的优势：**
   - **适应性：** 能够根据中间结果动态调整策略
   - **探索性：** 在未知环境中表现出色
   - **灵活性：** 不需要预先知道完整的解决方案路径

**选择建议：**
- 对于**定义明确、多步骤**的任务：优先考虑规划架构
- 对于**探索性、未知路径**的任务：优先考虑ReAct架构
- 对于**需要平衡效率和适应性的复杂场景**：可以考虑混合架构

理解这些权衡对系统设计者至关重要。为正确的问题选择正确的架构是构建有效和智能AI代理的关键技能。